In [1]:
# -*- coding: utf-8 -*-
# python3 example_cifar10_scnn.py
import sys
sys.path.append("..")
from importlib import import_module
import torch,time,os,random
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append("..")
import LIAF
from datasets.cifar10_dvs_simple import cifar10_DVS
from tensorboardX import SummaryWriter
from importlib import import_module
from LIAFnet.LIAFResNet import *
autocast = LIAF.autocast #统一autocast模式
bestacc = 0  # best test accuracy
################################ parameters ####################
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
for lr_index in range(1,10):
    for batch_index in range(1,4):
        task = 'Longwith_pretarin' + str(lr_index+batch_index*11)
        writer = SummaryWriter(comment=task)
        learning_rate = 1e-3 * lr_index
        batch_size  = 128 * batch_index
        num_epochs = 100
        timeWindows = 8
        names = 'dvs_cifar10'
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        train_path = '/data/CIFAR10-DVS-mini/train.mat' #r := raw string
        test_path =  '/data/CIFAR10-DVS-mini/test.mat' #TODO:input your oath

        train_dataset = cifar10_DVS(train_path,'r',timeWindows) #max= 25?
        test_dataset = cifar10_DVS(test_path,'r',timeWindows)

        train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                   batch_size=batch_size,
                                                   shuffle=True)

        test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                   batch_size=batch_size,
                                                   shuffle=False)

        TA.timeWindows = 8 #用了attention必须固定时间长度
        modules = import_module('LIAFnet.LIAFResNet_18')
        config  = modules.Config()
        workpath = os.path.abspath(os.getcwd())
        num_epochs = config.num_epochs
        accumulation = config.accumulation
        timeWindows = config.timeWindows
        config.cfgCnn = [2, 64, 7, True]
        config.dataSize = (42,42)
        config.actFun= LIAF.LIFactFun.apply
        epoch = 0
        
        start_epoch = 0  # start from epoch 0 or last checkpoint epoch
        training_iter = 0
        snn = LIAFResNet(config)
        checkpoint = torch.load('./45.pkl', map_location=torch.device('cpu'))
        snn.load_state_dict(checkpoint)
        snn.fc = nn.Linear(snn.cfgFc_[0],10)
        print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in snn.parameters())))

        snn.to(device)
        criterion = nn.CrossEntropyLoss()

        optimizer = torch.optim.Adam(snn.parameters(),
                        lr=learning_rate,
                        weight_decay =1e-4)


        lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
                            optimizer, 
                            milestones=[20,40,60], 
                            gamma=0.1, 
                            last_epoch=-1)
        def val(optimizer,snn,test_loader,test_dataset,batch_size,epoch):
            print('===> evaluating models...')
            snn.eval()
            correct = 0
            total = 0
            predicted = 0
            with torch.no_grad():
                for batch_idx, (inputs, labels) in enumerate(test_loader):
                    inputs = inputs.permute([0,1,4,2,3])
                    if ((batch_idx+1)<=len(test_dataset)//batch_size):
                        optimizer.zero_grad()
                        outputs = snn(inputs.to(device))

                        _ , labels = labels.max(1)
                        _ , predicted = outputs.cpu().max(1)
                        total += float(labels.size(0))
                        correct += float(predicted.eq(labels).sum())
            acc = 100. * float(correct) / float(total)
            print('================')
            print('val acc:',acc , '%  epoch:',epoch)
            print('================')
            writer.add_scalar('acc', acc, epoch)
            return acc

        save_folder = './saved_model'

        for epoch in range(num_epochs):
            # 新增2：设置sampler的epoch，DistributedSampler需要这个来维持各个进程之间的相同随机数种子
            #train_loader.sampler.set_epoch(epoch)
            snn.train()
            running_loss = 0
            start_time = time.time() 
            print('===> training models...')
            correct = 0.0
            total = 0.0
            torch.cuda.empty_cache()
            for i, (images, labels) in enumerate(train_loader):
                images = images.permute([0,1,4,2,3])
                if ((i+1)<=len(train_dataset)//batch_size):
                    with autocast():
                        outputs = snn(images.to(device)).cpu()
                        _,labels = labels.max(1)
                        _ , predict = outputs.max(1)
                        loss = criterion(outputs, labels)
                        correct += predict.eq(labels).sum()
                        total += float(predict.size(0))

                        loss /= accumulation
                        running_loss += loss.item()
                        loss.backward()

                    if (i+1)%accumulation == 0:
                        optimizer.step()
                        snn.zero_grad()
                        optimizer.zero_grad()

                    if (i+1)%(accumulation*18) == 0:
                        if not os.path.isdir(save_folder):
                            os.mkdir(save_folder)
                        train_acc =  100* correct / total
                        print('=====> Epoch [%d/%d], Step [%d/%d], Loss: %.5f , Acc: %.5f  '
                          %(epoch+start_epoch, num_epochs+start_epoch, i+1, len(train_dataset)//(batch_size),running_loss, train_acc))
                        writer.add_scalar('running Loss', running_loss, training_iter)
                        writer.add_scalar('running Acc', train_acc, training_iter)
                        print('bestacc:',bestacc)
                        correct = 0.0
                        total = 0.0
                        running_loss = 0
                training_iter +=1 
            torch.cuda.empty_cache()
            #evaluation
            acc = val(optimizer,snn,test_loader,test_dataset,batch_size,epoch)
            lr_scheduler.step()
            if not os.path.isdir(save_folder):
                os.mkdir(save_folder)
            if acc > bestacc:
                bestacc = acc
                print('===> Saving models...')
torch.save(snn.state_dict(),'./'+save_folder+'/violance_search'+str(int(bestacc))+'.pkl')

use torch.cuda.amp.autocast for fusion prcision training
torch.Size([9000, 2, 42, 42, 8]) torch.Size([9000, 10])
torch.Size([1000, 2, 42, 42, 8]) torch.Size([1000, 10])
Total number of paramerters in networks is 11185379  
===> training models...


/home/lyh/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


=====> Epoch [0/40], Step [18/70], Loss: 34.57988 , Acc: 30.12153  
bestacc: 0
=====> Epoch [0/40], Step [36/70], Loss: 29.84581 , Acc: 40.75521  
bestacc: 0
=====> Epoch [0/40], Step [54/70], Loss: 28.33179 , Acc: 44.44444  
bestacc: 0
===> evaluating models...
val acc: 31.919642857142858 %  epoch: 0
===> Saving models...
===> training models...
=====> Epoch [1/40], Step [18/70], Loss: 24.68986 , Acc: 51.43229  
bestacc: 31.919642857142858
=====> Epoch [1/40], Step [36/70], Loss: 24.60862 , Acc: 52.21354  
bestacc: 31.919642857142858
=====> Epoch [1/40], Step [54/70], Loss: 24.87879 , Acc: 51.69271  
bestacc: 31.919642857142858
===> evaluating models...
val acc: 42.075892857142854 %  epoch: 1
===> Saving models...
===> training models...
=====> Epoch [2/40], Step [18/70], Loss: 22.53141 , Acc: 57.07465  
bestacc: 42.075892857142854
=====> Epoch [2/40], Step [36/70], Loss: 23.46286 , Acc: 54.47049  
bestacc: 42.075892857142854
=====> Epoch [2/40], Step [54/70], Loss: 22.76369 , Acc: 55

=====> Epoch [20/40], Step [36/70], Loss: 6.95723 , Acc: 88.06424  
bestacc: 55.580357142857146
=====> Epoch [20/40], Step [54/70], Loss: 6.77786 , Acc: 87.54340  
bestacc: 55.580357142857146
===> evaluating models...
val acc: 62.05357142857143 %  epoch: 20
===> Saving models...
===> training models...
=====> Epoch [21/40], Step [18/70], Loss: 5.10614 , Acc: 92.01389  
bestacc: 62.05357142857143
=====> Epoch [21/40], Step [36/70], Loss: 5.00920 , Acc: 91.75347  
bestacc: 62.05357142857143
=====> Epoch [21/40], Step [54/70], Loss: 4.70480 , Acc: 92.40451  
bestacc: 62.05357142857143
===> evaluating models...
val acc: 63.504464285714285 %  epoch: 21
===> Saving models...
===> training models...
=====> Epoch [22/40], Step [18/70], Loss: 3.89002 , Acc: 94.05382  
bestacc: 63.504464285714285
=====> Epoch [22/40], Step [36/70], Loss: 3.74343 , Acc: 94.35764  
bestacc: 63.504464285714285
=====> Epoch [22/40], Step [54/70], Loss: 3.60655 , Acc: 94.14062  
bestacc: 63.504464285714285
===> evalu

=====> Epoch [2/40], Step [18/35], Loss: 22.38454 , Acc: 56.59722  
bestacc: 64.84375
===> evaluating models...
val acc: 20.182291666666668 %  epoch: 2
===> training models...
=====> Epoch [3/40], Step [18/35], Loss: 20.57713 , Acc: 60.28646  
bestacc: 64.84375
===> evaluating models...
val acc: 23.307291666666668 %  epoch: 3
===> training models...
=====> Epoch [4/40], Step [18/35], Loss: 19.14148 , Acc: 63.12934  
bestacc: 64.84375
===> evaluating models...
val acc: 28.515625 %  epoch: 4
===> training models...
=====> Epoch [5/40], Step [18/35], Loss: 17.65825 , Acc: 66.03732  
bestacc: 64.84375
===> evaluating models...
val acc: 37.760416666666664 %  epoch: 5
===> training models...
=====> Epoch [6/40], Step [18/35], Loss: 16.79572 , Acc: 67.46962  
bestacc: 64.84375
===> evaluating models...
val acc: 31.119791666666668 %  epoch: 6
===> training models...
=====> Epoch [7/40], Step [18/35], Loss: 15.40287 , Acc: 70.35590  
bestacc: 64.84375
===> evaluating models...
val acc: 36.19791

...

===> evaluating models...
val acc: 49.479166666666664 %  epoch: 29
===> training models...
=====> Epoch [30/40], Step [18/23], Loss: 16.32646 , Acc: 68.59086  
bestacc: 65.625
===> evaluating models...
val acc: 47.65625 %  epoch: 30
===> training models...
=====> Epoch [31/40], Step [18/23], Loss: 14.95858 , Acc: 71.62905  
bestacc: 65.625
===> evaluating models...
val acc: 49.739583333333336 %  epoch: 31
===> training models...
=====> Epoch [32/40], Step [18/23], Loss: 13.96670 , Acc: 73.50983  
bestacc: 65.625
===> evaluating models...
val acc: 51.041666666666664 %  epoch: 32
===> training models...
=====> Epoch [33/40], Step [18/23], Loss: 13.63403 , Acc: 73.71239  
bestacc: 65.625
===> evaluating models...
val acc: 47.65625 %  epoch: 33
===> training models...
=====> Epoch [34/40], Step [18/23], Loss: 12.86891 , Acc: 75.34722  
bestacc: 65.625
===> evaluating models...
val acc: 48.828125 %  epoch: 34
===> training models...
=====> Epoch [35/40], Step [18/23], Loss: 11.68631 , Acc: 

===> evaluating models...
val acc: 38.839285714285715 %  epoch: 15
===> training models...
=====> Epoch [16/40], Step [18/70], Loss: 26.25225 , Acc: 48.78472  
bestacc: 65.625
=====> Epoch [16/40], Step [36/70], Loss: 26.19663 , Acc: 47.09201  
bestacc: 65.625
=====> Epoch [16/40], Step [54/70], Loss: 26.63967 , Acc: 47.87326  
bestacc: 65.625
===> evaluating models...
val acc: 29.352678571428573 %  epoch: 16
===> training models...
=====> Epoch [17/40], Step [18/70], Loss: 26.19316 , Acc: 47.91667  
bestacc: 65.625
=====> Epoch [17/40], Step [36/70], Loss: 26.86109 , Acc: 47.09201  
bestacc: 65.625
=====> Epoch [17/40], Step [54/70], Loss: 26.25480 , Acc: 49.04514  
bestacc: 65.625
===> evaluating models...
val acc: 44.97767857142857 %  epoch: 17
===> training models...
=====> Epoch [18/40], Step [18/70], Loss: 25.37301 , Acc: 49.00174  
bestacc: 65.625
=====> Epoch [18/40], Step [36/70], Loss: 26.34181 , Acc: 48.39410  
bestacc: 65.625
=====> Epoch [18/40], Step [54/70], Loss: 26.431

=====> Epoch [37/40], Step [54/70], Loss: 18.37635 , Acc: 64.19271  
bestacc: 65.625
===> evaluating models...
val acc: 52.120535714285715 %  epoch: 37
===> training models...
=====> Epoch [38/40], Step [18/70], Loss: 17.62827 , Acc: 65.10416  
bestacc: 65.625
=====> Epoch [38/40], Step [36/70], Loss: 18.17616 , Acc: 64.40972  
bestacc: 65.625
=====> Epoch [38/40], Step [54/70], Loss: 16.80006 , Acc: 67.53472  
bestacc: 65.625
===> evaluating models...
val acc: 50.558035714285715 %  epoch: 38
===> training models...
=====> Epoch [39/40], Step [18/70], Loss: 16.86003 , Acc: 68.09896  
bestacc: 65.625
=====> Epoch [39/40], Step [36/70], Loss: 18.06155 , Acc: 64.45312  
bestacc: 65.625
=====> Epoch [39/40], Step [54/70], Loss: 18.19511 , Acc: 64.19271  
bestacc: 65.625
===> evaluating models...
val acc: 50.558035714285715 %  epoch: 39
torch.Size([9000, 2, 42, 42, 8]) torch.Size([9000, 10])
torch.Size([1000, 2, 42, 42, 8]) torch.Size([1000, 10])
Total number of paramerters in networks is 1

===> evaluating models...
val acc: 49.348958333333336 %  epoch: 34
===> training models...
=====> Epoch [35/40], Step [18/35], Loss: 14.07662 , Acc: 72.82986  
bestacc: 65.625
===> evaluating models...
val acc: 50.130208333333336 %  epoch: 35
===> training models...
=====> Epoch [36/40], Step [18/35], Loss: 13.64093 , Acc: 73.93663  
bestacc: 65.625
===> evaluating models...
val acc: 48.828125 %  epoch: 36
===> training models...
=====> Epoch [37/40], Step [18/35], Loss: 13.67242 , Acc: 73.43750  
bestacc: 65.625
===> evaluating models...
val acc: 49.21875 %  epoch: 37
===> training models...
=====> Epoch [38/40], Step [18/35], Loss: 12.59514 , Acc: 76.23698  
bestacc: 65.625
===> evaluating models...
val acc: 48.958333333333336 %  epoch: 38
===> training models...
=====> Epoch [39/40], Step [18/35], Loss: 12.92359 , Acc: 75.08681  
bestacc: 65.625
===> evaluating models...
val acc: 48.567708333333336 %  epoch: 39
torch.Size([9000, 2, 42, 42, 8]) torch.Size([9000, 10])
torch.Size([1000

===> evaluating models...
val acc: 49.739583333333336 %  epoch: 33
===> training models...
=====> Epoch [34/40], Step [18/23], Loss: 14.06017 , Acc: 72.98901  
bestacc: 65.625
===> evaluating models...
val acc: 49.088541666666664 %  epoch: 34
===> training models...
=====> Epoch [35/40], Step [18/23], Loss: 13.15102 , Acc: 75.24595  
bestacc: 65.625
===> evaluating models...
val acc: 50.0 %  epoch: 35
===> training models...
=====> Epoch [36/40], Step [18/23], Loss: 13.01617 , Acc: 74.85532  
bestacc: 65.625
===> evaluating models...
val acc: 48.828125 %  epoch: 36
===> training models...
=====> Epoch [37/40], Step [18/23], Loss: 11.78007 , Acc: 77.69097  
bestacc: 65.625
===> evaluating models...
val acc: 48.4375 %  epoch: 37
===> training models...
=====> Epoch [38/40], Step [18/23], Loss: 11.19798 , Acc: 78.93519  
bestacc: 65.625
===> evaluating models...
val acc: 47.526041666666664 %  epoch: 38
===> training models...
=====> Epoch [39/40], Step [18/23], Loss: 10.87810 , Acc: 79.74